In [2]:
import pandas as pd
# Import required libraries and dependencies

import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from sklearn.svm import SVC
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# SVC

In [3]:
attributes_df = pd.read_csv("../Resources/songAttributes_1999-2019.csv")
print(attributes_df.info())
attributes_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154931 entries, 0 to 154930
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        154931 non-null  int64  
 1   Acousticness      154931 non-null  float64
 2   Album             154931 non-null  object 
 3   Artist            154931 non-null  object 
 4   Danceability      154931 non-null  float64
 5   Duration          154931 non-null  int64  
 6   Energy            154931 non-null  float64
 7   Explicit          154931 non-null  bool   
 8   Instrumentalness  154931 non-null  float64
 9   Liveness          154931 non-null  float64
 10  Loudness          154931 non-null  float64
 11  Mode              154931 non-null  int64  
 12  Name              154931 non-null  object 
 13  Popularity        154931 non-null  int64  
 14  Speechiness       154931 non-null  float64
 15  Tempo             154931 non-null  float64
 16  TimeSignature     15

,Unnamed: 0,Acousticness,Album,Artist,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Name,Popularity,Speechiness,Tempo,TimeSignature,Valence
0,0,0.000728,Collective Soul (Deluxe Version),Collective Soul,0.520,234947,0.904,False,0.010300,0.0634,-5.030,1,Welcome All Again,35,0.0309,106.022,4,0.365
1,1,0.018200,Collective Soul (Deluxe Version),Collective Soul,0.581,239573,0.709,False,0.000664,0.1740,-4.909,1,Fuzzy,31,0.0282,120.027,4,0.408
2,2,0.000473,Collective Soul (Deluxe Version),Collective Soul,0.572,198400,0.918,False,0.000431,0.0977,-3.324,0,Dig,30,0.0559,144.061,4,0.370
3,3,0.000970,Collective Soul (Deluxe Version),Collective Soul,0.596,231453,0.661,False,0.000033,0.1130,-5.051,1,You,35,0.0254,111.975,4,0.183
4,4,0.000036,Collective Soul (Deluxe Version),Collective Soul,0.520,222520,0.808,False,0.000010,0.0800,-4.553,0,My Days,21,0.0318,92.721,4,0.666


In [4]:
attributes_scaled_arr = StandardScaler().fit_transform(
    attributes_df[[
        "Danceability",
        "Duration",
        "Energy",
        "Explicit",
        "Instrumentalness",
        "Liveness",
        "Popularity",
        "Loudness",
        "Mode"
    ]]
)

attributes_scaled_df= pd.DataFrame(attributes_scaled_arr, columns=[[
        "Danceability",
        "Duration",
        "Energy",
        "Explicit",
        "Instrumentalness",
        "Liveness",
        "Popularity",
        "Loudness",
        "Mode"
    ]])
attributes_scaled_df

,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Popularity,Loudness,Mode
0,-0.330792,0.020763,1.140056,-0.521167,-0.264008,-0.822735,0.893636,0.720162,0.681208
1,0.032455,0.059148,0.302125,-0.521167,-0.312271,-0.346070,0.651309,0.749236,0.681208
2,-0.021139,-0.282493,1.200215,-0.521167,-0.313438,-0.674908,0.590727,1.130082,-1.467981
3,0.121778,-0.008229,0.095865,-0.521167,-0.315429,-0.608968,0.893636,0.715116,0.681208
4,-0.330792,-0.082353,0.727536,-0.521167,-0.315549,-0.751192,0.045490,0.834776,-1.467981
...,...,...,...,...,...,...,...,...,...
154926,1.020962,-0.263134,0.044300,-0.521167,-0.315596,-0.354689,3.619819,0.434707,-1.467981
154927,-0.634490,-0.231819,-0.033047,-0.521167,-0.315596,-0.492603,3.256328,0.350368,-1.467981
154928,1.205563,-0.167097,-0.209227,-0.521167,-0.315596,-0.707232,3.316909,0.595696,0.681208
154929,0.895910,-0.349762,0.396661,-0.521167,-0.315580,-0.696026,3.195746,0.414043,-1.467981


In [5]:
import numpy as np
# attributes_scaled_df.describe()
x = -0.2
attributes_scaled_df['Popularity'] = np.where((attributes_scaled_df['Popularity'] > x) & attributes_scaled_df['Popularity'].notna(), 1, 0)


In [6]:
attributes_scaled_df["Popularity"].value_counts()

(Popularity,)
1                80626
0                74305
Name: count, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
data = attributes_scaled_df.copy()
data.drop(columns=["Popularity"],inplace=True)


C:\Users\Aspire-PC\AppData\Local\Temp\ipykernel_17552\2660290603.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  data.drop(columns=["Popularity"],inplace=True)


In [8]:
target = attributes_scaled_df["Popularity"]
target_names=["Popular","Not popular"]

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [9]:
from sklearn.svm import SVC
model = SVC(kernel='linear') # Search the meaning of the Kernel
model.fit(X_train, y_train)

c:\Users\Aspire-PC\anaconda3\envs\orange3\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear')

In [10]:
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.565


In [11]:
# Calculate the classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

     Popular       0.60      0.27      0.37     18568
 Not popular       0.55      0.84      0.67     20165

    accuracy                           0.56     38733
   macro avg       0.58      0.55      0.52     38733
weighted avg       0.58      0.56      0.53     38733



# CLEANING DATA

In [12]:
attributes_df

,Unnamed: 0,Acousticness,Album,Artist,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Name,Popularity,Speechiness,Tempo,TimeSignature,Valence
0,0,0.000728,Collective Soul (Deluxe Version),Collective Soul,0.520,234947,0.904,False,0.010300,0.0634,-5.030,1,Welcome All Again,35,0.0309,106.022,4,0.365
1,1,0.018200,Collective Soul (Deluxe Version),Collective Soul,0.581,239573,0.709,False,0.000664,0.1740,-4.909,1,Fuzzy,31,0.0282,120.027,4,0.408
2,2,0.000473,Collective Soul (Deluxe Version),Collective Soul,0.572,198400,0.918,False,0.000431,0.0977,-3.324,0,Dig,30,0.0559,144.061,4,0.370
3,3,0.000970,Collective Soul (Deluxe Version),Collective Soul,0.596,231453,0.661,False,0.000033,0.1130,-5.051,1,You,35,0.0254,111.975,4,0.183
4,4,0.000036,Collective Soul (Deluxe Version),Collective Soul,0.520,222520,0.808,False,0.000010,0.0800,-4.553,0,My Days,21,0.0318,92.721,4,0.666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154926,114,0.333000,No.6 Collaborations Project,Ed Sheeran,0.747,200733,0.649,False,0.000000,0.1720,-6.218,0,Nothing On You (feat. Paulo Londra & Dave),80,0.2190,88.011,4,0.840
154927,115,0.314000,No.6 Collaborations Project,Ed Sheeran,0.469,204507,0.631,False,0.000000,0.1400,-6.569,0,I Don't Want Your Money (feat. H.E.R.),74,0.0853,91.418,4,0.763
154928,116,0.467000,No.6 Collaborations Project,Ed Sheeran,0.778,212307,0.590,False,0.000000,0.0902,-5.548,1,1000 Nights (feat. Meek Mill & A Boogie Wit da...,75,0.2700,97.995,4,0.515
154929,117,0.110000,No.6 Collaborations Project,Ed Sheeran,0.726,190293,0.731,False,0.000003,0.0928,-6.304,0,Way To Break My Heart (feat. Skrillex),73,0.1590,170.125,4,0.430


In [13]:
categorical_df = attributes_df.drop(columns=["Acousticness","Unnamed: 0", "Danceability","Duration","Energy","Explicit","Instrumentalness","Liveness","Loudness","Mode","Name","Speechiness","Tempo", "TimeSignature", "Valence"])
categorical_df


,Album,Artist,Popularity
0,Collective Soul (Deluxe Version),Collective Soul,35
1,Collective Soul (Deluxe Version),Collective Soul,31
2,Collective Soul (Deluxe Version),Collective Soul,30
3,Collective Soul (Deluxe Version),Collective Soul,35
4,Collective Soul (Deluxe Version),Collective Soul,21
...,...,...,...
154926,No.6 Collaborations Project,Ed Sheeran,80
154927,No.6 Collaborations Project,Ed Sheeran,74
154928,No.6 Collaborations Project,Ed Sheeran,75
154929,No.6 Collaborations Project,Ed Sheeran,73


In [14]:
x = 1.5
categorical_df['Popularity'] = np.where((categorical_df['Popularity'] > x) & categorical_df['Popularity'].notna(), 1, 0)
categorical_df["Artist"].value_counts()

Artist
Oakenfold         1797
Willie Nelson     1586
Bing Crosby       1434
Cash Cash         1406
Elvis Presley     1312
                  ... 
Vitamin C           31
Teairra Mari        31
Julianne Hough      27
Wham!               27
Chris Lake          21
Name: count, Length: 989, dtype: int64

In [15]:
df_05 = pd.read_csv("../Resources/riaaSingleCerts_1999-2019.csv")
print(df_05.info())
df_05.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2601 entries, 0 to 2600
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   X           2601 non-null   int64 
 1   Name        2601 non-null   object
 2   Artist      2601 non-null   object
 3   RiaaStatus  2601 non-null   object
 4   Label       2601 non-null   object
dtypes: int64(1), object(4)
memory usage: 101.7+ KB
None


,X,Name,Artist,RiaaStatus,Label
0,0,Wish Wish,Dj Khaled,Gold,We the Best / Epic
1,1,Level Up,Ciara,1x Platinum,Beauty Marks Entertainment
2,2,Con Calma,Daddy Yankee,11x Diamond,Universal Music Latino / El Cartel Records
3,3,Wish you Were Gay,Billie Eilish,1x Platinum,Darkroom / Interscope Records
4,4,El Anillo,Jennifer Lopez,6x Multi-Platinum,Sony Latin


In [16]:
df_05.drop("X", axis=1, inplace=True)



In [132]:
merged_df = pd.merge(attributes_df, df_05, on=["Name", "Artist"], how ="inner")
merged_df.dropna()
merged_df.head()


,Unnamed: 0,Acousticness,Album,Artist,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Name,Popularity,Speechiness,Tempo,TimeSignature,Valence,RiaaStatus,Label
0,0,0.154000,All The Pain Money Can Buy,Fastball,0.653,257093,0.874,False,0.000014,0.307,-5.660,0,The Way,65,0.0341,120.840,4,0.902,Gold,Hollywood Records
1,38,0.178000,So Real,Mandy Moore,0.813,234133,0.836,False,0.000066,0.154,-4.230,1,Candy,10,0.0881,100.491,4,0.677,Gold,550 Music/Epic
2,25,0.435000,Ideal,Ideal,0.702,275173,0.520,False,0.000000,0.299,-8.581,1,Get Gone,35,0.0725,120.005,4,0.467,Gold,Virgin
3,48,0.000134,All The Good Sh**. 14 Solid Gold Hits (2000-2008),Sum 41,0.348,199493,0.951,False,0.004380,0.138,-5.026,0,The Hell Song,40,0.0771,173.409,4,0.672,Gold,Island Records
4,63,0.000140,All the Good Shit,Sum 41,0.352,199493,0.954,False,0.002710,0.092,-5.083,0,The Hell Song,29,0.0846,173.392,4,0.552,Gold,Island Records


In [133]:
pattern_01 = r'.*Atlantic.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_01: 'Atlantic'}, regex=True)

pattern_02 = r'.*Columbia.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_02: 'Columbia'}, regex=True)

pattern_03 = r'.*Def Jam.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_03: 'Def Jam'}, regex=True)

pattern_04 = r'.*Universal.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_04: 'Universal'}, regex=True)

pattern_05 = r'.*Warner Bros..*'
merged_df['Label'] = merged_df['Label'].replace({pattern_05: 'Warner Bros.'}, regex=True)

pattern_06 = r'.*Capitol.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_06: 'Capitol'}, regex=True)

pattern_07 = r'.*Rca.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_07: 'Rca'}, regex=True)

pattern_08 = r'.*Republic.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_08: 'Republic'}, regex=True)

pattern_09 = r'.*Island.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_09: 'Island'}, regex=True)

pattern_10 = r'.*Interscope.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_10: 'Interscope'}, regex=True)

pattern_11 = r'.*Hollywood.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_11: 'Hollywood'}, regex=True)

pattern_12 = r'.*Hollywood.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_12: 'Hollywood'}, regex=True)

pattern_13 = r'.*Arista.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_13: 'Arista'}, regex=True)

pattern_14 = r'.*Sony.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_14: 'Sony'}, regex=True)

pattern_15 = r'.*Mca.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_15: 'Mca'}, regex=True)

pattern_16 = r'.*Roc.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_16: 'Roc'}, regex=True)

pattern_17 = r'.*Warner.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_17: 'Warner'}, regex=True)

pattern_18 = r'.*Glassnote.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_18: 'Glassnote'}, regex=True)

pattern_19 = r'.*Epic.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_19: 'Epic'}, regex=True)

pattern_20 = r'.*Geffen.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_20: 'Geffen'}, regex=True)

pattern_21 = r'.*A&m.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_21: 'A&m'}, regex=True)

pattern_22 = r'.*Bna.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_22: 'Bna'}, regex=True)

pattern_23 = r'.*Virgin.*'
merged_df['Label'] = merged_df['Label'].replace({pattern_23: 'Virgin'}, regex=True)

print(merged_df["Label"].value_counts())
print(merged_df["Label"].nunique())



Label
Atlantic                                           180
Interscope                                         113
Columbia                                           103
Def Jam                                            100
Republic                                            85
Rca                                                 84
Capitol                                             71
Warner                                              70
Universal                                           66
Big Machine Records, Inc.                           58
Arista                                              57
Epic                                                53
Island                                              26
Hollywood                                           20
Mercury                                             16
Reprise                                             15
Jive                                                15
A&m                                                 15
Sony

In [134]:
labels_to_replace = list(merged_df["Label"].value_counts()[merged_df["Label"].value_counts() < 3].index)

# Replace in dataframe
for i in labels_to_replace:
    merged_df['Label'] = merged_df['Label'].replace(i,"Other")
    
# Check to make sure binning was successful
merged_df['Label'].value_counts()

Label
Atlantic                     180
Interscope                   113
Columbia                     103
Def Jam                      100
Republic                      85
Rca                           84
Capitol                       71
Warner                        70
Universal                     66
Other                         61
Big Machine Records, Inc.     58
Arista                        57
Epic                          53
Island                        26
Hollywood                     20
Mercury                       16
Jive                          15
A&m                           15
Reprise                       15
Sony                          13
Bna                           12
Mca                           11
Elektra                       10
Geffen                        10
Sire                           9
Wind-Up Records                9
Laface                         9
Glassnote                      8
Curb Records                   8
Big Machine Records            8
j Re